In [1]:
import pandas as pd
from gensim.models.fasttext import FastText
import tensorflow as tf
import numpy as np

Label Read In 

In [2]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df['cat_num'] = df['category'].astype('category').cat.codes
labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Loading input matrix

In [3]:
trainInp = np.load('trainChar.npy')
valInp = np.load('valChar.npy')
embed = np.load('embedChar.npy')
embed = embed.astype(np.float64)

In [4]:
learning_rate = 0.001
batch_size = 64

In [5]:
tf.reset_default_graph()

Batching and creating iterators

In [6]:
train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [7]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

In [8]:
embedding

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 547, 8) dtype=float64>

In [9]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, 547, 8, 1) dtype=float64>

Implementing looped convolution

In [10]:
pooled_outputs = []
filter_sizes = [3,4,5]
embedding_size = embed.shape[1]
num_filters1 = 32
num_filters2 = 64
max_length = 547
for filter_size in filter_sizes:
    filter_shape1 = [filter_size, embedding_size, 1, num_filters1]
    W1 = tf.Variable(tf.truncated_normal(filter_shape1, stddev=0.1), name='W1')
    b1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]), name = 'b1')
    conv1 = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W1,tf.float64),
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1')
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, tf.cast(b1,tf.float64)), name="relu")
    #pooled_outputs.append(conv1)
    
    filter_shape2 = [filter_size, embedding_size, num_filters1, num_filters2]
    W2 = tf.Variable(tf.truncated_normal(filter_shape2, stddev=0.1), name='W2')
    b2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]), name = 'b2')    
    conv2 = tf.nn.conv2d(
        relu1,
        tf.cast(W2,tf.float64),
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv2')
    
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, tf.cast(b2,tf.float64)), name="relu")
    pooled = tf.nn.max_pool(
        relu2,
        ksize=[1, max_length, 8, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)    

In [11]:
filter_shape2

[5, 8, 32, 64]

In [12]:
pooled_outputs

[<tf.Tensor 'pool:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_1:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_2:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_3:0' shape=(?, 1, 1, 64) dtype=float64>]

Combining separate convolutional layers into 1 feed forward input

In [13]:
num_filters_total = num_filters2 * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [14]:
combined_flat

<tf.Tensor 'Reshape:0' shape=(?, 256) dtype=float64>

Adding dense layers

In [15]:
conn = tf.layers.dense(combined_flat, 100, activation = 'relu')
conn2 = tf.layers.dense(conn, len(set(df.cat_num)))

In [16]:
conn

<tf.Tensor 'dense/Relu:0' shape=(?, 100) dtype=float64>

Implementing cross entropy, loss, and optimization

In [17]:
n_epochs = 10
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = conn2)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Prediction setup

In [18]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [19]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)

Average loss epoch 0: 1.2050409391511783
Accuracy 0: 0.5587797619047619
Average val loss epoch 0: 0.8698438643326386
Val_Accuracy 0: 0.7021276595744681
Average loss epoch 1: 0.7396061454774175
Accuracy 1: 0.7366071428571429
Average val loss epoch 1: 0.734126664315569
Val_Accuracy 1: 0.7460106382978723


KeyboardInterrupt: 

Calculating accuracy for train

In [73]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))

0.7728494623655914